In [79]:
import numpy as np
import pandas as pd
import random
from typing import List, Tuple
from itertools import combinations
from geopy.distance import geodesic

POPULATION_SIZE = 200              # Population size
OFFSPRING_SIZE = 50                # Number of offspring per generation
MAX_GENERATIONS = 1000             # Maximum number of generations
MUTATION_RATE = 0.99               # Mutation probability

# Loading cities and building the distance matrix
CITIES = pd.read_csv('./italy.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km

# Parameters for the genetic algorithm
NUM_CITIES = len(CITIES)           # Number of cities


## Fitness

In [80]:
# Function to calculate the path cost (total length)
def path_cost(path: List[int]) -> float:
    return sum(DIST_MATRIX[path[i], path[(i + 1) % NUM_CITIES]] for i in range(NUM_CITIES))

# Function to calculate the path cost (total length)
def path_cost_circular(path: List[int]) -> float:
    # Calculate the total distance, including the return from the last city to the first
    total_cost = 0.0
    for i in range(len(path)):
        total_cost += DIST_MATRIX[path[i], path[(i + 1) % len(path)]]  # Include last to first
    return total_cost

# Fitness function: inverse of the path length
def fitness(individual: List[int]) -> float:
    return -path_cost(individual)

def fitness_circular(individual: List[int]) -> float:
    return -path_cost_circular(individual)


## Inizialization

In [81]:
def KKN(start_city: str) -> Tuple[List[int], float]:
    start_city_index = CITIES[CITIES['name'] == start_city].index[0]
    path = [start_city_index]
    visited = set(path)
    current_city = start_city_index
    while len(visited) < len(CITIES):
        nearest_distance = float('inf')
        # Find the nearest cities
        for city_index in range(len(CITIES)):
            if city_index not in visited:
                distance = DIST_MATRIX[current_city, city_index]
                if distance < nearest_distance:
                    nearest_distance = distance
                    nearest_city = city_index
        path.append(nearest_city)
        visited.add(nearest_city)
        current_city = nearest_city
    path.append(start_city_index)  # return to the starting point
    # Local optimization: 2-opt
    path = tweak(path)
    return path

def tweak(path: List[int]) -> List[int]:
    """ Performs 2-opt optimization on the given path. """
    improved = True
    while improved:
        improved = False
        for i in range(1, len(path) - 2):
            for j in range(i + 1, len(path)):
                if j - i == 1:  # Avoid reversing two consecutive points
                    continue
                new_path = path[:]
                new_path[i:j] = reversed(path[i:j])  # Reverse the subsequence
                if path_cost(new_path) < path_cost(path):
                    path = new_path
                    improved = True
    return path

def initialize_population(size: int, start_city: str ) -> List[List[int]]:
    # Find the index of the starting city
    start_city_index = CITIES[CITIES['name'] == start_city].index[0]
    
    # List of city indices excluding the starting one
    other_cities = [i for i in range(NUM_CITIES) if i != start_city_index]
    
    # Generate the population by fixing the starting and ending city
    population = []
    for _ in range(size):
        individual = [start_city_index]  
        individual += list(np.random.permutation(other_cities))  # Permutation of other cities
        individual.append(start_city_index)  # Add the starting city as the last element
        population.append(individual)
    
    return population

def initialize_population_with_greedy_and_shuffle(size: int, start_city: str) -> List[List[int]]:
    # Find the index of the starting city
    start_city_index = CITIES[CITIES['name'] == start_city].index[0]

    # List of city indices excluding the starting one
    other_cities = [i for i in range(len(CITIES)) if i != start_city_index]

    # Initial population
    population = []

    # Generate 5 greedy paths and add them to the population
    for _ in range(5):
        greedy_path = KKN(start_city)
        population.append(greedy_path)

    # Generate size-5 random paths and add them to the population
    for _ in range(size - 5):
        individual = [start_city_index]  # Starting city as the first element
        individual += list(np.random.permutation(other_cities))  # Permutation of other cities
        individual.append(start_city_index)  # Starting city as the last element
        population.append(individual)

    # Shuffle the population randomly
    random.shuffle(population)

    return population

from collections import deque

def initialize_population_buffer(size: int) -> deque:
    # Initialize the circular buffer with a fixed maximum size
    population_buffer = deque(maxlen=size)
    
    # List of all city indices
    all_cities = list(range(NUM_CITIES))  # Supponendo che NUM_CITIES rappresenti il numero totale di città
    
    # Generate the population by creating random permutations of all cities
    for _ in range(size):
        individual = list(np.random.permutation(all_cities))  # Permutazione di tutte le città
        population_buffer.append(individual)
    
    return population_buffer



## Parent selection function

In [82]:
# Parent selection using tournament
def parent_selection(population: List[List[int]]) -> List[int]:
    tournament = random.sample(population, 5)
    tournament.sort(key=fitness, reverse=True)
    return tournament[0]
def parent_selection_circular(population: List[List[int]]) -> List[int]:
    tournament = random.sample(population, 5)
    tournament.sort(key=fitness_circular, reverse=True)
    return tournament[0]


## Crossover functions

In [83]:
def inver_over_crossover(parent1: List[int], parent2: List[int]) -> List[int]:
    # Copy the first parent's path to start creating the child
    
    # Create copies of parents excluding the start and end cities
    modified_parent1 = parent1[1:-1]  # Exclude the first and last element
    modified_parent2 = parent2[1:-1]  # Exclude the first and last element

    child = modified_parent1[:]
    # Select a random city (gene) within the path, excluding start and end
    gene_index = random.randint(0, len(modified_parent1) - 1)
    gene = child[gene_index]
    
    # Find the position of the next city in the second parent
    next_index = (modified_parent2.index(gene) + 1) % len(modified_parent2)
    next_city = modified_parent2[next_index]
    
    next_index = child.index(next_city)

    # Perform the inversion of the subsequence between gene_index and next_index
    if gene_index < next_index:
        child[gene_index:next_index+1] = reversed(child[gene_index:next_index + 1])
    else:
        child[next_index:gene_index+1] = reversed(child[next_index:gene_index + 1])

    # Now we need to add the starting and ending cities from parent1[0] and parent1[-1]
    child.insert(0, parent1[0])
    child.append(parent1[0])
    
    return child
def inver_over_crossover_circular(parent1: List[int], parent2: List[int]) -> List[int]:
    # Create copies of parents without altering their structure
    modified_parent1 = parent1[:]  # Include all cities
    modified_parent2 = parent2[:]  # Include all cities

    child = modified_parent1[:]  # Start with a copy of the first parent
    
    # Select a random city (gene) within the path
    gene_index = random.randint(0, len(modified_parent1) - 1)
    gene = child[gene_index]

    # Find the position of the next city in the second parent
    next_index = (modified_parent2.index(gene) + 1) % len(modified_parent2)
    next_city = modified_parent2[next_index]

    next_index = child.index(next_city)

    # Perform the inversion of the subsequence between gene_index and next_index
    if gene_index < next_index:
        child[gene_index:next_index + 1] = reversed(child[gene_index:next_index + 1])
    else:
        child[next_index:gene_index + 1] = reversed(child[next_index:gene_index + 1])

    return child

# Crossover operator: Ordinal Crossover (OX)
def crossover(parent1: List[int], parent2: List[int]) -> List[int]:
    # Set the first and last element as fixed (start/end city)
    start, end = sorted(random.sample(range(1, NUM_CITIES - 1), 2))
    child = [parent1[0]] + [None] * (NUM_CITIES - 2) + [parent1[0]]
    
    # Copy the selected segment from the first parent
    child[start:end] = parent1[start:end]
    
    # Complete the path with cities from the second parent, avoiding duplicates
    position = end
    for city in parent2:
        if city not in child:
            if position >= NUM_CITIES - 1:
                position = 1  # Restart from the second element to avoid first and last
            child[position] = city
            position += 1

    return child

# Crossover operator: Ordinal Crossover (OX)
def crossover_circular(parent1: List[int], parent2: List[int]) -> List[int]:
    # Set a segment of the path to be copied from the first parent
    start, end = sorted(random.sample(range(len(parent1)), 2))  # Include anche il primo e l'ultimo elemento
    child = [None] * len(parent1)  # Inizializza il figlio con valori None
    
    # Copy the selected segment from the first parent
    child[start:end] = parent1[start:end]
    
    # Complete the path with cities from the second parent, avoiding duplicates
    position = end
    for city in parent2:
        if city not in child:
            if position >= len(child):
                position = 0  # Restart from the beginning if we reach the end
            child[position] = city
            position += 1

    return child



## Mutation functions

In [84]:
# Mutation operator: Randomly swaps two cities
def mutate(individual: List[int]) -> List[int]:
    if random.random() < MUTATION_RATE:
        # Select two cities to swap, excluding the starting and ending city
        i, j = random.sample(range(1, NUM_CITIES - 1), 2)
        individual[i], individual[j] = individual[j], individual[i]
    return individual



def inversion_mutation(individual: List[int]) -> List[int]:
    # Ensures the starting and ending city are fixed
    if random.random() < MUTATION_RATE:
        start_city = individual[0]
        end_city = individual[-1]
        
        # Select two random indices among the internal cities (1 to len(individual) - 2)
        idx1, idx2 = sorted(random.sample(range(1, len(individual) - 1), 2))
        
        # Reverse the sequence between idx1 and idx2
        individual[idx1:idx2 + 1] = reversed(individual[idx1:idx2 + 1])
        
        # Restore the starting and ending city
        individual[0] = start_city
        individual[-1] = end_city
    
    return individual

def inversion_mutation_circular(individual: List[int]) -> List[int]:
    # Allows inversion without fixing start and end
    if random.random() < MUTATION_RATE:
        # Select two random indices among all cities
        idx1, idx2 = sorted(random.sample(range(len(individual)), 2))
        
        # Reverse the sequence between idx1 and idx2
        individual[idx1:idx2 + 1] = reversed(individual[idx1:idx2 + 1])

    return individual




In [85]:
def genetic_algorithm(population: List[List[int]], max_generations: int, offspring_size: int) -> List[int]:
 
    for generation in range(max_generations):
        # Offspring creation
        offspring = []
        for _ in range(offspring_size):
            parent1 = parent_selection(population)
            parent2 = parent_selection(population)
            child = inver_over_crossover(parent1, parent2)
            child = inversion_mutation(child)
            offspring.append(child)
        
        # Fitness evaluation of the offspring
        population.extend(offspring)
        population.sort(key=fitness, reverse=True)
        population = population[:len(population) - offspring_size]  # Keeps the best ones for the initial size

        # Print every 100 generations
        if generation % 100 == 0:
            print(f"Generation {generation}, Best Path Cost: {path_cost(population[0]):.2f} km")

    # Final result
    best_individual = population[0]
    print("Best path found:", [CITIES['name'][i] for i in best_individual])
    print(f"Cost of best path: {path_cost(best_individual):.2f} km")

    return best_individual


In [86]:
def genetic_algorithm_circular(population: deque, max_generations: int, offspring_size: int) -> List[int]:
 
    for generation in range(max_generations):
        # Offspring creation
        offspring = []
        for _ in range(offspring_size):
            parent1 = parent_selection_circular(population)
            parent2 = parent_selection_circular(population)
            child = inver_over_crossover_circular(parent1, parent2)  # Usa la funzione di crossover aggiornata
            child = inversion_mutation_circular(child)     # Usa la funzione di mutazione aggiornata
            offspring.append(child)
        
        # Fitness evaluation of the offspring
        population.extend(offspring)
        population = deque(sorted(population, key=fitness_circular, reverse=True)[:len(population) - offspring_size])  # Keeps the best ones for the initial size

        # Print every 100 generations
        if generation % 100 == 0:
            print(f"Generation {generation}, Best Path Cost: {path_cost_circular(population[0]):.2f} km")

    # Final result
    best_individual = population[0]
    print("Best path found:", [CITIES['name'][i] for i in best_individual])
    print(f"Cost of best path: {path_cost_circular(best_individual):.2f} km")

    return best_individual


In [87]:
population = initialize_population(POPULATION_SIZE, start_city="Syracuse")
population2 = initialize_population_with_greedy_and_shuffle(POPULATION_SIZE, start_city="Syracuse")
genetic_algorithm(population, MAX_GENERATIONS, OFFSPRING_SIZE)
genetic_algorithm(population2, MAX_GENERATIONS, OFFSPRING_SIZE)
population3 = initialize_population_buffer(POPULATION_SIZE)
genetic_algorithm_circular(population3, MAX_GENERATIONS, OFFSPRING_SIZE)

None

Generation 0, Best Path Cost: 15429.41 km
Generation 100, Best Path Cost: 7868.86 km
Generation 200, Best Path Cost: 6137.69 km
Generation 300, Best Path Cost: 5431.22 km
Generation 400, Best Path Cost: 4824.00 km
Generation 500, Best Path Cost: 4734.60 km
Generation 600, Best Path Cost: 4504.54 km
Generation 700, Best Path Cost: 4432.54 km
Generation 800, Best Path Cost: 4344.18 km
Generation 900, Best Path Cost: 4313.00 km
Best path found: ['Syracuse', 'Catania', 'Reggio di Calabria', 'Messina', 'Taranto', 'Bari', 'Andria', 'Foggia', 'Salerno', 'Naples', 'Giugliano in Campania', 'Pescara', 'Latina', 'Rome', 'Terni', 'Perugia', 'Ancona', 'Rimini', 'Forlì', 'Ravenna', 'Ferrara', 'Florence', 'Prato', 'Bologna', 'Modena', "Reggio nell'Emilia", 'Parma', 'Piacenza', 'Brescia', 'Verona', 'Vicenza', 'Padua', 'Venice', 'Trieste', 'Bolzano', 'Trento', 'Bergamo', 'Monza', 'Milan', 'Novara', 'Turin', 'Genoa', 'Leghorn', 'Sassari', 'Cagliari', 'Palermo', 'Syracuse']
Cost of best path: 4313.00 km


In [89]:
population = initialize_population(POPULATION_SIZE, start_city="Terni")
population2 = initialize_population_with_greedy_and_shuffle(POPULATION_SIZE, start_city="Terni")
genetic_algorithm(population, MAX_GENERATIONS, OFFSPRING_SIZE)
genetic_algorithm(population2, MAX_GENERATIONS, OFFSPRING_SIZE)
None

Generation 0, Best Path Cost: 14897.47 km
Generation 100, Best Path Cost: 6998.23 km
Generation 200, Best Path Cost: 5506.24 km
Generation 300, Best Path Cost: 5069.25 km
Generation 400, Best Path Cost: 4805.05 km
Generation 500, Best Path Cost: 4682.82 km
Generation 600, Best Path Cost: 4415.50 km
Generation 700, Best Path Cost: 4409.05 km
Generation 800, Best Path Cost: 4366.31 km
Generation 900, Best Path Cost: 4284.05 km
Best path found: ['Terni', 'Rome', 'Latina', 'Giugliano in Campania', 'Naples', 'Salerno', 'Foggia', 'Andria', 'Bari', 'Taranto', 'Reggio di Calabria', 'Messina', 'Catania', 'Syracuse', 'Palermo', 'Cagliari', 'Sassari', 'Leghorn', 'Prato', 'Florence', 'Bologna', 'Modena', "Reggio nell'Emilia", 'Parma', 'Genoa', 'Turin', 'Novara', 'Milan', 'Monza', 'Bergamo', 'Piacenza', 'Brescia', 'Verona', 'Trento', 'Bolzano', 'Vicenza', 'Padua', 'Trieste', 'Venice', 'Ferrara', 'Forlì', 'Ravenna', 'Rimini', 'Ancona', 'Pescara', 'Perugia', 'Terni']
Cost of best path: 4284.05 km
Gen